<img src="../static/logo.png" alt="datio" style="width: 200px "align="right"/>

## SAS PROC 1

proc sort data=diaria.Tsfaucno out=mdv_prealtas (keep=cod_persona cod_ofici fec_alta xti_origen  
where=(fec_alta>=&fecha_SEMANA_INI and fec_alta<=&fecha_SEMANA_FIN and xti_origen eq 'O')   
rename (COD_OFICIPR=COD_OFICI))
nodupkey;  
by cod_persona;  
run;

#### Read data and register as temporal table

In [7]:
import pyspark
from pyspark.sql.context import SQLContext
sc = pyspark.SparkContext('local[*]')
sqlContext = SQLContext(sc)
dataPath = "../data/tsfaucno_muestra.csv"
sqlContext.read.format('com.databricks.spark.csv')\
.options(header='true').option("inferSchema", "true").load(dataPath)\
.registerTempTable("Tsfaucno")


ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-2-f21f09b8d1c5>:4 

NOTE: To work over a dataframe with sql sentence it is necessary registerthe given DataFrame as a temporary table in the catalog.

Temporary tables exist only during the lifetime of this instance of SQLContext.

#### Show registered tables in catalog

In [8]:
sqlContext.sql("SHOW TABLES").collect()

[Row(tableName='tsfaucno', isTemporary=True)]

In [9]:
import datetime
fecha_SEMANA_INI = datetime.date(2015, 1, 21).strftime("%Y-%m-%d %X")
fecha_SEMANA_FIN = datetime.date(2015, 1, 28).strftime("%Y-%m-%d %X")

In [10]:
sqlContext.sql("select * from Tsfaucno").count()

1000

In [ ]:
#Query
query = "SELECT cod_persona, cod_ofici as cod_oficipr \
FROM Tsfaucno where fec_alta>= '" + fecha_SEMANA_INI  + "' and \
     fec_alta<='" + fecha_SEMANA_FIN + "' and xti_origen='O'"

sqlContext.sql(query)\
.registerTempTable("mdv_prealtas")

In [ ]:
sqlContext.sql("select * from mdv_prealtas").collect()

## SAS PROC 2  

data ingresos_semanal (keep= cod_persona tenencia_ingresos_semanal);  
set mdv_prealtas;  
tenencia_ingresos_semanal=1;  
area_recibos=13;  
run;  

In [ ]:
from pyspark.sql.functions import lit
sqlContext.sql("Select cod_persona \
FROM mdv_prealtas")\
.withColumn("tenencia_ingresos_semanal",lit('1'))\
.withColumn("area_recibos",lit('13'))\
.registerTempTable("ingresos_semanal")